In [34]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'itg-bpma-gbl-ww-dv' # Project ID inserted based on the query results selected to explore
location = 'EU' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [126]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [36]:
#Define Client Chanel_type and Product
client_name = 'BOOTS'
Channel_type = 'Online'
Product_ean = '800897813727'

In [37]:
# Define parameters for the filter
params = {
    'client_name': client_name,
    'ean_code': Product_ean
}

query = """
SELECT *
FROM bpma_ds_c2_exposed_eu_dv.sellout_and_promo_event_pivoted_V1
Where
client = @client_name AND
ean = @ean_code AND global_channel_type='Online'
"""

# Execute the query and read the result into a Pandas DataFrame
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter('client_name', 'STRING', params['client_name']),
        bigquery.ScalarQueryParameter('ean_code', 'STRING', params['ean_code'])
    ]
)

result_df = client.query(query, job_config=job_config).to_dataframe()

In [38]:
result_df.head()

,ean,country_id,global_channel_type,client,sold_units,sold_value_loc,start_date,end_date,mid_date,is_promo,...,DEFENSIVE ON Before,SHOPPER INVESTMENT ON Before,RETURNS During,STD COST OF SALES AND VARIANCES Before,CONSO NET SALES Before,PHYSICAL DISTRIBUTION During,NG NET SALES Incremental,ACTIVE SUPPORT OFF Incremental,INVOICED SALES Before,hierarchy_is_ean
0,800897813727,GB,Online,BOOTS,112.000000000,887.30,2021-11-14,2021-11-20,2021-11-17,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
1,800897813727,GB,Online,BOOTS,99.000000000,789.45,2021-05-30,2021-06-05,2021-06-02,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
2,800897813727,GB,Online,BOOTS,82.000000000,528.68,2020-01-19,2020-01-25,2020-01-22,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
3,800897813727,GB,Online,BOOTS,146.000000000,979.24,2024-03-10,2024-03-16,2024-03-13,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
4,800897813727,GB,Online,BOOTS,135.000000000,997.10,2022-02-06,2022-02-12,2022-02-09,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>


In [39]:
result_df.drop_duplicates(inplace=True)

In [40]:
# Supprimer les lignes où 'id_event' est null et 'is_promo' est True
result_df = result_df.drop(result_df[(result_df['event_id'].isnull()) & (result_df['is_promo'])].index)

In [41]:
result_df['start_date'].unique().shape[0]

222

In [42]:
events_columns_to_delete = [ 'OPERATIONAL_SUB_AXE_LABEL', 'event_id', 'event_start_date', 'event_end_date','country_id','promo_cluster_event',
                            'promo_cluster_event_ean', 'event_status', 'event_description', 'ean_event_id', 'hierarchy_is_ean']


columns_relevant_to_product = ['hierarchy_code', 'client', 'ean'
                               ]

In [43]:
df = result_df.drop(events_columns_to_delete+columns_relevant_to_product, axis=1)

In [44]:
#let's eliminate tactics and subtatics for further analysis
#data = df.drop(['sub_tactic', 'tactic', 'depth_of_discount_range', 'price_range'], axis=1)

In [45]:
# Function to convert percentage range strings to numeric average
def convert_to_average(range_str):
    if range_str != None :

      low, high = map(lambda x: float(x.replace('%', '')), range_str.split('-'))
      return (low + high) / 2
    return None

# Applying the conversion to the 'depth_of_discount_range' column
df['avg_discount'] = df['depth_of_discount_range'].apply(convert_to_average)

In [46]:
pnls = ['DEFENSIVE OFF Incremental', 'DEFENSIVE ON EXCL ROLLBACK During', 'NG NET SALES During', 'PREPACK DISPLAYS During', 'ACTIVE SUPPORT OFF CUST AGREM Incremental',
 'SELL OUT VALUE Before', 'INTERNATIONAL CUSTOMER INVESTMENT Before', 'GROSS SALES Incremental', 'SERVICES Before', 'DEFENSIVE ON Incremental',
 'SERVICES During', 'ACTIVE SUPPORT OFF CUST AGREM Before', 'ACTIVE SUPPORT OFF EXCL CUST AGREM Before', 'FINANCE OFF Before', 'SHOPPER INVESTMENT OFF Before',
 'ACTIVE SUPPORT OFF EXCL CUST AGREM Incremental', 'ACTIVE SUPPORT OFF CUST AGREM During', 'TOTAL COST During', 'FINANCE OFF During',
 'GROSS SALES Before', 'ACTIVE SUPPORT OFF During', 'TOTAL COST Before', 'INVOICED SALES During', 'INVOICED SALES Incremental', 'CLEARANCES AND RETURNS Before',
 'RETURNS Incremental', 'SHOPPER INVESTMENT OFF During', 'CLEARANCES AND RETURNS Incremental', 'PNL CHECK Before', 'SERVICES Incremental', 'ROLLBACK Incremental',
 'ACTIVE SUPPORT ON Incremental', 'ROLLBACK During', 'STD COST OF SALES AND VARIANCES During', 'EFFICIENCY ON Incremental', 'COST OF SALES Before',
 'GROSS SALES During', 'THEORETICAL SELLOUT VALUE Before', 'INTERNATIONAL CUSTOMER INVESTMENT During', 'ACTIVE SUPPORT OFF Before', 'GROWTH INCENTIVE Incremental',
 'CLEARANCES AND RETURNS During', 'EFFICIENCY OFF During', 'DEFENSIVE OFF During', 'SELL OUT VALUE Incremental', 'EFFICIENCY OFF Incremental',
 'PNL CHECK Incremental', 'EFFICIENCY ON Before', 'ROYAL TA MS Before', 'PHYSICAL DISTRIBUTION Before', 'OBSOLETE SLOW MOVING RETURNS Before',
 'EFFICIENCY ON During', 'SHOPPER INVESTMENT ON During', 'DEFENSIVE ON During', 'UNITS Incremental', 'FINANCE OFF Incremental', 'INTERNATIONAL CUSTOMER INVESTMENT Incremental',
 'SPECIAL OPS COST During', 'SHOPPER INVESTMENT ON Incremental', 'CLEARANCES During', 'GROSS MARGIN Incremental', 'CLEARANCES Incremental',
 'CONSO NET SALES Incremental', 'NG NET SALES Before', 'RETURNS Before', 'UNITS During', 'OBSOLETE SLOW MOVING RETURNS During', 'TOTAL COST Incremental',
 'CONSO NET SALES During', 'GROSS MARGIN During', 'DEFENSIVE ON EXCL ROLLBACK Before', 'COST OF SALES During', 'ACTIVE SUPPORT OFF EXCL CUST AGREM During',
 'ACTIVE SUPPORT ON During', 'ROLLBACK Before', 'CLEARANCES Before', 'EFFICIENCY OFF Before', 'UNITS Before', 'SELL OUT VALUE During', 'PNL CHECK During',
 'COST OF SALES Incremental', 'GROWTH INCENTIVE During', 'PREPACK DISPLAYS Before', 'ROYAL TA MS During', 'SPECIAL OPS COST Before',
 'DEFENSIVE OFF Before', 'ACTIVE SUPPORT ON Before', 'GROWTH INCENTIVE Before', 'DEFENSIVE ON EXCL ROLLBACK Incremental', 'GROSS MARGIN Before',
 'SHOPPER INVESTMENT OFF Incremental', 'DEFENSIVE ON Before', 'SHOPPER INVESTMENT ON Before', 'RETURNS During', 'STD COST OF SALES AND VARIANCES Before',
 'CONSO NET SALES Before', 'PHYSICAL DISTRIBUTION During', 'NG NET SALES Incremental', 'ACTIVE SUPPORT OFF Incremental', 'INVOICED SALES Before']

In [47]:
data=df.copy()

In [48]:
data = df.drop(pnls, axis=1)

In [49]:
data.drop('depth_of_discount_range',axis=1, inplace=True)

In [50]:
data.drop_duplicates(inplace=True)

In [51]:
data.tail(5)

,global_channel_type,sold_units,sold_value_loc,start_date,end_date,mid_date,is_promo,week,year,Final_ISP,sub_axis,Filled_Final_ISP,estimated_price,seasonality_index,price_range,tactic,sub_tactic,avg_discount
441,Online,307.000000000,1940.00,2023-05-28,2023-06-03,2023-05-31,True,22,2023,8.0,Face Makeup,8.0,6.319218,1.016535,6.3,"Promotion OCA , Shopper Discount","Online , Save , Site Fee",22.5
443,Online,131.000000000,1036.46,2023-04-30,2023-05-06,2023-05-03,True,18,2023,8.0,Face Makeup,8.0,7.911908,1.037166,7.7,"Promotion OCA , Shopper Discount","Online , Save , Site Fee",2.5
444,Online,152.000000000,915.77,2023-09-10,2023-09-16,2023-09-13,True,37,2023,8.0,Face Makeup,8.0,6.024803,0.967239,6.0,"MultiBuy , Promotion OCA","3 for 2 , Online , Site Fee",27.5
454,Online,250.000000000,1282.60,2023-10-22,2023-10-28,2023-10-25,True,43,2023,8.0,Face Makeup,8.0,5.130400,0.988779,5.5,"Promotion OCA , Shopper Discount","Online , Save , Site Fee",32.5
456,Online,145.000000000,1010.09,2024-02-04,2024-02-10,2024-02-07,True,6,2024,8.0,Face Makeup,8.0,6.966138,0.994004,6.5,"Promotion OCA , Shopper Discount","Save , Site Fee",17.5


In [52]:
#data.columns

In [53]:
data.groupby('start_date').size().reset_index(name='counts')

,start_date,counts
0,2019-12-29,1
1,2020-01-05,1
2,2020-01-12,1
3,2020-01-19,1
4,2020-01-26,1
...,...,...
216,2024-02-18,1
217,2024-02-25,1
218,2024-03-03,1
219,2024-03-10,2


In [54]:
date_data = data[data['start_date']==pd.Timestamp('2023-07-30')]
date_data.shape

(2, 18)

In [55]:
for col in data.columns.to_list():
  count = date_data[col].unique().shape[0]
  if count>1:
    print(col, count )

price_range 2


In [56]:
date_data[['sold_units', 'avg_discount', 'price_range', 'tactic', 'sub_tactic']]

,sold_units,avg_discount,price_range,tactic,sub_tactic
318,177.000000000,7.5,7.4,"Promotion OCA , Shopper Discount","Online , Save , Site Fee"
319,177.000000000,7.5,7.5,"Promotion OCA , Shopper Discount","Online , Save , Site Fee"


In [57]:
result = data.groupby('start_date').agg({
    'avg_discount': lambda x: x.mean() if len(x) > 1 else x.iloc[0],
    'price_range': lambda x: x.mean() if len(x) > 1 else x.iloc[0],
    'tactic': lambda x: ', '.join(x) if len(x) > 1 else x.iloc[0],
    'sub_tactic': lambda x: ', '.join(x) if len(x) > 1 else x.iloc[0]
}).reset_index()

In [58]:
result.tail()

,start_date,avg_discount,price_range,tactic,sub_tactic
216,2024-02-18,17.5,6.7,"Promotion OCA , Shopper Discount","Online , Save , Site Fee"
217,2024-02-25,17.5,6.7,"Promotion OCA , Shopper Discount","Online , Save , Site Fee"
218,2024-03-03,17.5,6.7,"Promotion OCA , Shopper Discount","Online , Save , Site Fee"
219,2024-03-10,15.0,6.9,"Promotion OCA , Shopper Discount, Promotion OC...","Online , Save , Site Fee, Online , Save , Site..."
220,2024-03-17,12.5,7.1,"Promotion OCA , Shopper Discount","Online , Save , Site Fee"


In [59]:
# Fusionner les résultats agrégés avec le DataFrame original en utilisant 'start_date'
merged_data = pd.merge(data, result, on='start_date', suffixes=('', '_agg'))

# Sélectionner les colonnes et remplacer les valeurs originales par les agrégées si nécessaire
for column in ['avg_discount', 'price_range', 'tactic', 'sub_tactic']:
    merged_data[column] = merged_data[column + '_agg']
    merged_data.drop(column + '_agg', axis=1, inplace=True)

In [60]:
merged_data.drop_duplicates(inplace=True)

In [61]:
merged_data.groupby('start_date').size().reset_index(name='counts')

,start_date,counts
0,2019-12-29,1
1,2020-01-05,1
2,2020-01-12,1
3,2020-01-19,1
4,2020-01-26,1
...,...,...
216,2024-02-18,1
217,2024-02-25,1
218,2024-03-03,1
219,2024-03-10,1


In [62]:
def remove_duplicates_from_string(input_string):
  if input_string==None : return None
  unique_items = set(input_string.split(', '))
  cleaned_string = ', '.join(sorted(unique_items))
  return cleaned_string

merged_data['tactic'] = merged_data['tactic'].apply(remove_duplicates_from_string)
merged_data['sub_tactic'] = merged_data['sub_tactic'].apply(remove_duplicates_from_string)

In [63]:
merged_data.head()

,global_channel_type,sold_units,sold_value_loc,start_date,end_date,mid_date,is_promo,week,year,Final_ISP,sub_axis,Filled_Final_ISP,estimated_price,seasonality_index,price_range,tactic,sub_tactic,avg_discount
0,Online,112.000000000,887.30,2021-11-14,2021-11-20,2021-11-17,False,46,2021,NaN,Face Makeup,8.0,7.922321,1.038199,NaN,None,None,NaN
1,Online,99.000000000,789.45,2021-05-30,2021-06-05,2021-06-02,False,22,2021,NaN,Face Makeup,8.0,7.974242,1.074688,NaN,None,None,NaN
2,Online,82.000000000,528.68,2020-01-19,2020-01-25,2020-01-22,False,3,2020,NaN,Face Makeup,8.0,6.447317,1.000000,NaN,None,None,NaN
3,Online,135.000000000,997.10,2022-02-06,2022-02-12,2022-02-09,False,6,2022,8.0,Face Makeup,8.0,7.385926,1.043246,NaN,None,None,NaN
4,Online,192.000000000,1517.45,2021-08-01,2021-08-07,2021-08-04,False,31,2021,NaN,Face Makeup,8.0,7.903385,1.031296,NaN,None,None,NaN


In [118]:
data = merged_data.copy()

In [119]:
unecessary_columns = ['global_channel_type', 'start_date', 'mid_date', 'Final_ISP', 'sub_axis', 'Filled_Final_ISP']
data.drop(unecessary_columns, axis=1, inplace=True)

In [120]:
data = data.sort_values(by='end_date')

In [121]:
# Replace nulls in 'price_range' and 'avrg_discount' with zero
data.loc[(data['is_promo'] == False) & (data['price_range'].isnull()), 'price_range'] = 0
data.loc[(data['is_promo'] == False) & (data['avg_discount'].isnull()), 'avg_discount'] = 0
data.loc[(data['is_promo'] == False) & (data['estimated_price'].isnull()), 'estimated_price'] = 0
# Replace nulls in 'tactic' and 'subtactic' with empty strings
data.loc[(data['is_promo'] == False) & (data['tactic'].isnull()), 'tactic'] = ''
data.loc[(data['is_promo'] == False) & (data['sub_tactic'].isnull()), 'sub_tactic'] = ''

In [122]:
data['sold_units'] = pd.to_numeric(data['sold_units'], errors='coerce')

In [123]:
data.head()

,sold_units,sold_value_loc,end_date,is_promo,week,year,estimated_price,seasonality_index,price_range,tactic,sub_tactic,avg_discount
19,89.0,617.95,2020-01-04,False,52,2019,6.943258,1.0,0.0,,,0.0
87,89.0,562.91,2020-01-11,False,1,2020,6.324831,1.0,0.0,,,0.0
100,67.0,417.47,2020-01-18,False,2,2020,6.230896,1.0,0.0,,,0.0
2,82.0,528.68,2020-01-25,False,3,2020,6.447317,1.0,0.0,,,0.0
104,96.0,593.42,2020-02-01,False,4,2020,6.181458,1.0,0.0,,,0.0


In [124]:
data.isnull().sum()

sold_units           0
sold_value_loc       0
end_date             0
is_promo             0
week                 0
year                 0
estimated_price      0
seasonality_index    0
price_range          0
tactic               0
sub_tactic           0
avg_discount         0
dtype: int64

In [127]:
numeric_data = data.select_dtypes(include=[np.number])

In [129]:
numeric_data.corr()

,sold_units,sold_value_loc,week,year,estimated_price,seasonality_index,price_range,avg_discount
sold_units,1.000000,0.944361,0.211422,0.004811,-0.422645,-0.118065,0.026121,0.140345
sold_value_loc,0.944361,1.000000,0.258911,0.111751,-0.183349,-0.019144,0.080321,0.131906
week,0.211422,0.258911,1.000000,-0.151650,0.035787,0.034865,-0.133266,0.013437
year,0.004811,0.111751,-0.151650,1.000000,0.252688,0.142106,0.800410,0.674841
estimated_price,-0.422645,-0.183349,0.035787,0.252688,1.000000,0.367248,0.066157,-0.170212
seasonality_index,-0.118065,-0.019144,0.034865,0.142106,0.367248,1.000000,0.042650,-0.249159
price_range,0.026121,0.080321,-0.133266,0.800410,0.066157,0.042650,1.000000,0.763723
avg_discount,0.140345,0.131906,0.013437,0.674841,-0.170212,-0.249159,0.763723,1.000000


In [134]:
data_m = data.copy()
data_m['is_promo'] = data_m['is_promo'].astype(int)

In [135]:
data_m['is_promo']

19     0
87     0
100    0
2      0
104    0
      ..
194    1
214    1
219    1
157    1
221    1
Name: is_promo, Length: 221, dtype: int64

In [136]:
numeric_data = data_m.select_dtypes(include=[np.number])
numeric_data.corr()

,sold_units,sold_value_loc,is_promo,week,year,estimated_price,seasonality_index,price_range,avg_discount
sold_units,1.000000,0.944361,0.045855,0.211422,0.004811,-0.422645,-0.118065,0.026121,0.140345
sold_value_loc,0.944361,1.000000,0.091183,0.258911,0.111751,-0.183349,-0.019144,0.080321,0.131906
is_promo,0.045855,0.091183,1.000000,-0.115425,0.807171,0.028254,-0.005762,0.994141,0.828608
week,0.211422,0.258911,-0.115425,1.000000,-0.151650,0.035787,0.034865,-0.133266,0.013437
year,0.004811,0.111751,0.807171,-0.151650,1.000000,0.252688,0.142106,0.800410,0.674841
estimated_price,-0.422645,-0.183349,0.028254,0.035787,0.252688,1.000000,0.367248,0.066157,-0.170212
seasonality_index,-0.118065,-0.019144,-0.005762,0.034865,0.142106,0.367248,1.000000,0.042650,-0.249159
price_range,0.026121,0.080321,0.994141,-0.133266,0.800410,0.066157,0.042650,1.000000,0.763723
avg_discount,0.140345,0.131906,0.828608,0.013437,0.674841,-0.170212,-0.249159,0.763723,1.000000


In [137]:
data_m.info()

<class 'pandas.core.frame.DataFrame'>
Index: 221 entries, 19 to 221
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sold_units         221 non-null    float64
 1   sold_value_loc     221 non-null    float64
 2   end_date           221 non-null    dbdate 
 3   is_promo           221 non-null    int64  
 4   week               221 non-null    Int64  
 5   year               221 non-null    Int64  
 6   estimated_price    221 non-null    float64
 7   seasonality_index  221 non-null    float64
 8   price_range        221 non-null    float64
 9   tactic             221 non-null    object 
 10  sub_tactic         221 non-null    object 
 11  avg_discount       221 non-null    float64
dtypes: Int64(2), dbdate(1), float64(6), int64(1), object(2)
memory usage: 22.9+ KB


In [ ]:
sold_unit, end_date, /estimated_price, seasonality_index, avg_discount, sub_tactic